### Importing libraries

In [ ]:
import os,sys,kfp,zipfile,json
from typing import Callable
from kfp.components._structures import MetadataSpec
import kfp
import string,random,tempfile,datetime
import kfp.components as kfplc
from typing import NamedTuple
from dkube.sdk import *

### DKUBE-SDK

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

### Create Resources

In [ ]:
def artifactmgr(user: str, token: str):
    import json
    from dkube.sdk.api import DkubeApi
    from dkube.sdk.rsrcs import DkubeCode
    from dkube.sdk.rsrcs import DkubeDataset
    from dkube.sdk.rsrcs import DkubeModel


    api = DkubeApi(token=token)

    # Block to create the code resources here
    # Copy/Paste below lines to create N num of code resources from here
    name = "mnist-1"
    code = DkubeCode(user, name=name)
    code.update_git_details(
        'https://github.com/oneconvergence/dkube-examples-internal/tree/2.0.6/tensorflow/classification/mnist/digits/classifier/program')
    api.create_code(code)

    # Create the input dataset resources here
    name = "mnist-ds"
    dataset = DkubeDataset(user, name=name)
    dataset.update_dataset_source(source='git')
    dataset.update_git_details(
        'https://github.com/oneconvergence/dkube-examples-internal/tree/2.0.6/tensorflow/classification/mnist/digits/classifier/data')
    api.create_dataset(dataset)

    # Create the output dataset resources here.
    output_datasets = ["output-ds-1", "output-ds-2"]
    for name in output_datasets:
        dataset = DkubeDataset(user, name=name)
        dataset.update_dataset_source(source='dvs')
        api.create_dataset(dataset)

    # Create the output model resources here
    output_models = ["output-model-1", "output-model-2"]
    for name in output_models:
        model = DkubeModel(user, name=name)
        model.update_model_source(source='dvs')
        api.create_model(model)


In [ ]:
def componentize(fn: Callable, name: str, desc: str,
                 image: str, annotations: dict, labels: dict):
    labels.update({'wfid': '{{workflow.uid}}', 'runid': '{{pod.name}}'})
    md = MetadataSpec(annotations=annotations, labels=labels)

    fn._component_human_name = name
    fn._component_description = desc

    cfunc = kfp.components.create_component_from_func(
        fn,
        base_image=image,
    )
    cfunc.component_spec.metadata = md
    envs: Mapping[str, str] = {'pipeline': 'true',
                               'wfid': '{{workflow.uid}}', 'runid': '{{pod.name}}'}
    cfunc.component_spec.implementation.container.env = envs
    cfunc.component_spec.save("artifactmgr.yaml")
    return cfunc

### Create a Component

In [ ]:

dkube_artifact_op = componentize(artifactmgr,
                                 "dkube_artifactmgr",
                                 "DKube artifact resource manager",
                                 "ocdr/dkube_launcher:viz",
                                 {
                                     'platform': 'Dkube'
                                 },
                                 {
                                     'platform': 'Dkube',
                                     'logger': 'dkubepl',
                                     'dkube.garbagecollect': 'false',
                                     'dkube.garbagecollect.policy':
                                     'all'
                                 })

### Loading the component from above created yaml

In [ ]:
dkube_artifact_op = kfp.components.load_component_from_file("artifactmgr.yaml")

client = kfp.Client(existing_token=token)

### Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='pl-resources',
    description='create resources'
)
def resources_pipeline(token):
    
        rs_pl  = dkube_artifact_op(username, token)

### Create Run

In [ ]:
client.create_run_from_pipeline_func(resources_pipeline, arguments={"token":token})

### Upload Pipeline

In [ ]:
def generate(name):
    timestamp = datetime.datetime.now().strftime("%H%M%S")
    return "{}-{}{}".format(
        name, timestamp, "".join([random.choice(string.digits) for n in range(4)])
    )
name = generate("resource-pl")
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(resources_pipeline, f.name)
client.upload_pipeline(f.name, pipeline_name=name)